In [1]:
from flask import Flask
import eventlet.wsgi
import eventlet
import socketio
from PIL import Image
from io import BytesIO
import base64
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model

sio = socketio.Server()
app = Flask(__name__)

Using TensorFlow backend.


In [2]:
model = load_model('data/beta_simulator_windows/mymodel.h5')
# model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [3]:
def process_image(img):
    #160, 320, 3 => 60,80,3
    return img[10:130:2, ::4, :]

In [ ]:
@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        speed = float(data["speed"])
        image_str = data["image"]
        
        decoded = base64.b64decode(image_str)
        image = Image.open(BytesIO(decoded))
        image_array = np.asarray(image)
#         plt.imshow(image_array)
#         plt.show()
#         print(image_array.shape)
        
#         steering_angle = 0.0  #angle -1 to 1. (left right)
#         throttle = 0.0   #0 - 1.0 speed
        img = process_image(image_array)
        img_batch = np.expand_dims(img, axis=0)
        
        steering_angle = float(model.predict(img_batch))
        
        throttle = 0.15
        if speed < 15:
            throttle = 0.5
        if speed > 17:
            throttle = -0.1
        
        send_control(steering_angle, throttle)
    else:
        sio.emit('manual', data={}, skip_sid=True)

def send_control(steering_angle, throttle):
    sio.emit("steer", data={'steering_angle': str(steering_angle),
                            'throttle': str(throttle) }, skip_sid=True)

app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(11524) wsgi starting up on http://0.0.0.0:4567
(11524) accepted ('127.0.0.1', 58051)
127.0.0.1 - - [18/Nov/2019 22:05:41] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 67.783448
(11524) accepted ('127.0.0.1', 58064)
(11524) accepted ('127.0.0.1', 58067)
(11524) accepted ('127.0.0.1', 58068)
